In [1]:
import pandas as pd

In [2]:
team_stats = pd.read_csv('TeamStatistics.csv')
#player_stats = pd.read_csv('PlayerStatistics.csv')
game_stats = pd.read_csv('Games.csv')

/tmp/ipykernel_98/3537571684.py:3: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  game_stats = pd.read_csv('Games.csv')


In [4]:
team_stats.head()

,gameId,gameDate,teamCity,teamName,teamId,opponentTeamCity,opponentTeamName,opponentTeamId,home,win,...,leadChanges,pointsFastBreak,pointsFromTurnovers,pointsInThePaint,pointsSecondChance,timesTied,timeoutsRemaining,seasonWins,seasonLosses,coachId
0,22500122,2025-10-27T22:30:00Z,Los Angeles,Lakers,1610612747,Portland,Trail Blazers,1610612757,1,0,...,9.0,19.0,30.0,54.0,16.0,6.0,1.0,2.0,2.0,NaN
1,22500122,2025-10-27T22:30:00Z,Portland,Trail Blazers,1610612757,Los Angeles,Lakers,1610612747,0,1,...,9.0,9.0,28.0,62.0,35.0,6.0,1.0,2.0,2.0,NaN
2,22500121,2025-10-27T22:00:00Z,Golden State,Warriors,1610612744,Memphis,Grizzlies,1610612763,1,1,...,6.0,12.0,21.0,48.0,11.0,4.0,0.0,3.0,1.0,NaN
3,22500121,2025-10-27T22:00:00Z,Memphis,Grizzlies,1610612763,Golden State,Warriors,1610612744,0,0,...,6.0,20.0,23.0,48.0,15.0,4.0,0.0,2.0,2.0,NaN
4,22500008,2025-10-27T21:30:00Z,Denver,Nuggets,1610612743,Minnesota,Timberwolves,1610612750,0,1,...,4.0,10.0,24.0,56.0,15.0,3.0,0.0,2.0,1.0,NaN


In [5]:
# Suppose gameDate is in YYYY-MM-DD format
team_stats['gameDate'] = pd.to_datetime(team_stats['gameDate'], format='mixed', utc=True)
team_stats['season'] = pd.to_datetime(team_stats['gameDate'])

def get_season_year(date):
    if date.month >= 7:  # July or later → season starts this year
        return date.year
    else:                # Before July → season started the previous year
        return date.year - 1

team_stats = team_stats.merge(
    game_stats[['gameId', 'gameType']],  # columns to bring over
    on='gameId',
    how='left'  # keep all rows from team_stats
)

team_stats['season'] = team_stats['gameDate'].apply(get_season_year)

In [40]:
# Separate out playoff and regular season games
regular_season_team_stats = team_stats[team_stats['gameType'] == 'Regular Season']
playoff_team_stats = team_stats[team_stats['gameType'] == 'Playoffs']

In [41]:
# Basic aggregations
regular_season_team = (
    regular_season_team_stats.groupby(['season', 'teamId', 'teamName'])
    .agg(
        games_played=('gameId', 'count'),
        wins=('win', 'sum'),
        losses=('win', lambda x: (1 - x).sum()),
        avg_points=('teamScore', 'mean'),
        avg_opponent_points=('opponentScore', 'mean'),
        avg_point_diff=('plusMinusPoints', 'mean'),
        fg_pct=('fieldGoalsPercentage', 'mean'),
        three_pct=('threePointersPercentage', 'mean'),
        ft_pct=('freeThrowsPercentage', 'mean'),
        assists=('assists', 'mean'),
        rebounds=('reboundsTotal', 'mean'),
        turnovers=('turnovers', 'mean'),
        steals=('steals', 'mean'),
        blocks=('blocks', 'mean'),
        fastbreak_pts=('pointsFastBreak', 'mean'),
        paint_pts=('pointsInThePaint', 'mean'),
        second_chance_pts=('pointsSecondChance', 'mean'),
        points_from_to=('pointsFromTurnovers', 'mean'),
        biggest_lead=('biggestLead', 'mean'),
        times_tied=('timesTied', 'mean'),
        lead_changes=('leadChanges', 'mean')
    )
    .reset_index()
)

playoff_season_team = (
    playoff_team_stats[playoff_team_stats['gameType'] == 'Playoffs']  # filter just to playoff games
    .groupby(['season', 'teamId', 'teamName'])
    .agg(
        playoff_games=('gameId', 'count'),
        playoff_wins=('win', 'sum'),
        playoff_losses=('win', lambda x: (1 - x).sum())
    )
    .reset_index()
)

# Derived features
regular_season_team['win_pct'] = regular_season_team['wins'] / regular_season_team['games_played']
regular_season_team['avg_margin'] = regular_season_team['avg_points'] - regular_season_team['avg_opponent_points']

numeric_cols = regular_season_team.select_dtypes(include='number').columns
regular_season_team[numeric_cols] = regular_season_team[numeric_cols].round(2)

In [48]:
# Merge regular season and playoff summaries
team_stats_full = pd.merge(
    regular_season_team,
    playoff_season_team,
    on=['season', 'teamId', 'teamName'],
    how='left'  # keep all regular-season teams, even if they didn't make playoffs
)

team_stats_full[['playoff_games', 'playoff_wins', 'playoff_losses']] = (
    team_stats_full[['playoff_games', 'playoff_wins', 'playoff_losses']].fillna(0)
)

In [49]:
team_stats_full_2000_on = team_stats_full[team_stats_full['season'] >= 2000]

In [50]:
team_stats_full_2000_on

,season,teamId,teamName,games_played,wins,losses,avg_points,avg_opponent_points,avg_point_diff,fg_pct,...,second_chance_pts,points_from_to,biggest_lead,times_tied,lead_changes,win_pct,avg_margin,playoff_games,playoff_wins,playoff_losses
911,2000,1610612737,Hawks,82,25,57,90.96,96.17,-5.00,0.43,...,NaN,NaN,NaN,NaN,NaN,0.30,-5.21,0.0,0.0,0.0
912,2000,1610612738,Celtics,82,36,46,94.62,96.76,-2.13,0.43,...,NaN,NaN,NaN,NaN,NaN,0.44,-2.13,0.0,0.0,0.0
913,2000,1610612739,Cavaliers,82,30,52,92.21,96.45,-4.24,0.44,...,NaN,NaN,NaN,NaN,NaN,0.37,-4.24,0.0,0.0,0.0
914,2000,1610612741,Bulls,82,15,67,87.57,96.67,-9.10,0.43,...,NaN,NaN,NaN,NaN,NaN,0.18,-9.10,0.0,0.0,0.0
915,2000,1610612742,Mavericks,82,53,29,100.48,96.20,4.28,0.46,...,NaN,NaN,NaN,NaN,NaN,0.65,4.28,10.0,4.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1652,2024,1610612762,Jazz,82,16,66,110.33,119.87,-9.65,0.45,...,14.68,13.13,9.27,4.33,6.01,0.20,-9.54,0.0,0.0,0.0
1653,2024,1610612763,Grizzlies,82,48,34,121.71,116.85,4.90,0.48,...,16.33,18.49,15.94,4.80,6.63,0.59,4.85,4.0,0.0,4.0
1654,2024,1610612764,Wizards,82,19,63,106.63,118.88,-12.40,0.44,...,12.67,14.72,7.35,4.49,6.37,0.23,-12.24,0.0,0.0,0.0
1655,2024,1610612765,Pistons,82,44,38,115.50,113.60,1.90,0.48,...,14.91,19.05,13.20,4.99,6.51,0.54,1.90,6.0,2.0,4.0


In [51]:
season_team_2024 = team_stats_full[team_stats_full['season'] == 2024]

In [52]:
season_team_2024.sort_values(by='wins', ascending=False).head(30)

,season,teamId,teamName,games_played,wins,losses,avg_points,avg_opponent_points,avg_point_diff,fg_pct,...,second_chance_pts,points_from_to,biggest_lead,times_tied,lead_changes,win_pct,avg_margin,playoff_games,playoff_wins,playoff_losses
1650,2024,1610612760,Thunder,83,68,15,120.02,107.51,12.52,0.48,...,14.70,21.75,21.10,4.20,5.83,0.82,12.52,23.0,16.0,7.0
1629,2024,1610612739,Cavaliers,82,64,18,121.94,112.40,9.54,0.49,...,13.78,18.95,18.54,4.41,6.58,0.78,9.54,9.0,5.0,4.0
1628,2024,1610612738,Celtics,82,61,21,116.27,107.16,9.11,0.46,...,14.25,15.53,18.33,4.68,6.56,0.74,9.11,11.0,6.0,5.0
1635,2024,1610612745,Rockets,81,51,30,114.47,109.95,4.52,0.45,...,18.24,17.51,15.48,4.81,7.25,0.63,4.52,7.0,3.0,4.0
1642,2024,1610612752,Knicks,82,51,31,115.78,111.68,4.10,0.49,...,14.33,19.21,16.06,4.49,6.21,0.62,4.10,18.0,10.0,8.0
1633,2024,1610612743,Nuggets,82,50,32,120.76,116.87,3.89,0.51,...,14.37,16.98,14.98,4.59,5.76,0.61,3.89,14.0,7.0,7.0
1644,2024,1610612754,Pacers,82,50,32,117.37,115.13,2.23,0.49,...,11.22,18.28,13.51,5.48,7.13,0.61,2.23,23.0,15.0,8.0
1640,2024,1610612750,Timberwolves,82,49,33,114.29,109.29,5.00,0.47,...,15.10,17.74,14.52,5.02,7.07,0.60,5.00,15.0,9.0,6.0
1637,2024,1610612747,Lakers,80,49,31,113.45,112.36,1.09,0.48,...,12.30,16.11,14.40,3.90,6.38,0.61,1.09,5.0,1.0,4.0
1636,2024,1610612746,Clippers,81,49,32,112.75,108.47,4.28,0.48,...,12.85,19.43,15.25,4.77,6.81,0.60,4.28,7.0,3.0,4.0


In [53]:
season_team_2024.columns

Index(['season', 'teamId', 'teamName', 'games_played', 'wins', 'losses',
       'avg_points', 'avg_opponent_points', 'avg_point_diff', 'fg_pct',
       'three_pct', 'ft_pct', 'assists', 'rebounds', 'turnovers', 'steals',
       'blocks', 'fastbreak_pts', 'paint_pts', 'second_chance_pts',
       'points_from_to', 'biggest_lead', 'times_tied', 'lead_changes',
       'win_pct', 'avg_margin', 'playoff_games', 'playoff_wins',
       'playoff_losses'],
      dtype='object')